In [83]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from bertopic import BERTopic

from sentence_transformers import SentenceTransformer
from nltk import pos_tag
import sklearn.manifold
import random
random.seed(42)

import yaml
from yaml.loader import SafeLoader

import re
import sys
sys.path.append('./mylib')
#from sentiment_process import *
import global_materials
import datetime

stopwords = global_materials.stopwords
exclude_stopwords = {"app"}
stopwords -= exclude_stopwords

In [84]:
TRAIN_ROOT = './train_data/'

if not os.path.exists("model"):
    os.mkdir("model")
if not os.path.exists("train_data"):
    os.mkdir("train_data")

In [85]:
# # Internal review
# file = "4_internal_EN_2022-09-26.csv"
# df_int = pd.read_csv(TRAIN_ROOT+file,index_col=False, sep='\t')
# print(df_int.columns.values)
# df_int = df_int[['review_en','rating','negative_word']]

# # external review
# file = "4_dataApp_EN_2022-09-23.csv"
# df_ext = pd.read_csv(TRAIN_ROOT+file,index_col=False, sep='\t')
# print(df_ext.columns.values)
# df_ext = df_ext[['review_en','rating','negative_word']]

# df_train = pd.concat([df_int, df_ext])
# print(df_train.shape)
# df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub(r'[^\w\s,]','', str(x)))
# df_train['review_en'].values

# # get Negative reviews
# df_train = df_train[(df_train['rating']<=3)|(df_train['negative_word']>1)]

# #df_train['review_en'].values[0:5]

# # Only train model with negative review or 
# df_train['review_wcount'] = df_train['review_en'].apply(lambda x: len(x.split()))
# #df_train.groupby('review_wcount')['review'].count().sort_values(ascending=False)
# df_train = df_train[df_train['review_wcount']>5]

In [86]:
########################################################################
# Train by data of internal review and external review having label
#    - Only train with negative sentiment data
########################################################################
# Internal review
file = "../sentiment_analysis_internal_review/data/train/5_dataApp_EN_clean_vader.csv" #4_internal_EN_2022-09-26.csv"
df_int = pd.read_csv(file,index_col=False)
print(df_int.columns.values)
df_int = df_int[['review_en','label']]

# external review
file = "../sentiment_analysis_external_review/data/train/5_dataApp_EN_clean_vader.csv" #"4_dataApp_EN_2022-09-23.csv"
df_ext = pd.read_csv(file,index_col=False)
print(df_ext.columns.values)
df_ext = df_ext[['review_en','label']]

df_train = pd.concat([df_int, df_ext])
print(df_train.shape)
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub(r'[^\w\s,]','', str(x)))
df_train['review_en'].values

df_train = df_train[df_train['label']==1]

# df_train['review_wcount'] = df_train['review_en'].apply(lambda x: len(x.split()))
# df_train = df_train[df_train['review_wcount']>5]

df_train['review_en'].values[0:5]


['SESSION_ID' 'DEVICE_ID' 'USER_ID' 'username' 'RATING_TYPE' 'rating'
 'review' 'date' 'RATING_DATE' 'review_adj' 'ymdhms' 'negative_word'
 'review_en' 'positive_word' 'review_clean' 'review_empty' 'vader_pos'
 'vader_neg' 'vader_neu' 'vader_compound' 'label_rating' 'flag_posword'
 'label_negword_pre' 'label_negword' 'label_vader' 'label_sum' 'label']
['username' 'review' 'date' 'm_y' 'rating' 'duplicate' 'review_adj'
 'ymdhms' 'negative_word' 'review_en' 'review_clean' 'review_empty'
 'positive_word' 'vader_pos' 'vader_neg' 'vader_neu' 'vader_compound'
 'label_rating' 'flag_posword' 'label_negword_pre' 'label_negword'
 'label_vader' 'label_sum' 'label']
(10510, 2)


array(['why cant use coins app', 'I cant see the card info',
       'the application time item reported an error',
       'The card item in the myvib app has an error and cannot be viewed',
       'error cant pay with coins'], dtype=object)

In [87]:
# !pip install --upgrade pip
# !pip install --upgrade numpy
# !pip install --upgrade sentence_transformers
# !conda install -c conda-forge hdbscan --y
# !pip install bokeh
# !pip install --upgrade bertopic[visualization]

In [88]:
# # Keep Noun and Adj only
# IGNORE_LIST = ['is','were','was','have','has','had','should','can','did','vib','vcb','vietcombank','oh','ah','uh','ooh','don','please','i','okay','db']
# def get_noun_adj(sentence):
#     #return ' '.join([pos[0] for pos in pos_tag(sentence.split()) if ((pos[1][0]=='N')|(pos[1][0]=='J')|(pos[1][0]=='V')|(pos[1][0]=='R'))&(pos[0] not in IGNORE_LIST)])
#     return ' '.join([pos[0] for pos in pos_tag(sentence.split()) if ((pos[1][0]=='N'))&(pos[0] not in IGNORE_LIST)])
# # def get_noun_adj(sentence):
# #     return ' '.join([pos[0] for pos in pos_tag(sentence.split()) if (pos[0] not in IGNORE_LIST)])


In [89]:
#texts = df_pred['review_en'].values.tolist()
df_train['review_en'] = df_train['review_en'].apply(lambda x: str(x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('\'ll',' will', x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('don\'t','do not', x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('can\'t','can not', x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('hasn\'t','has not', x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('wasn\'t','was not', x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('won\'t','will not', x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('cant','can not', x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('dont','do not', x))
df_train['review_en'] = df_train['review_en'].apply(lambda x: re.sub('hadn\'t','had not', x))
# #-----------------------------------------
# # Lemmatizer
# from nltk import pos_tag
# from nltk.stem import WordNetLemmatizer
    
# texts = [' '.join([WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tag(text.split())]) for text in texts] 

print(df_train['review_en'].values[:3])



['why can not use coins app' 'I can not see the card info'
 'the application time item reported an error']


In [90]:
# from bertopic.backend import WordDocEmbedder
# import gensim.downloader as api
# from sentence_transformers import SentenceTransformer

# # Word embedding model
# ft = api.load('fasttext-wiki-news-subwords-300')

# # Document embedding model
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# # Create a model that uses both language models and pass it through BERTopic
# word_doc_embedder = WordDocEmbedder(embedding_model=embedding_model, word_embedding_model=ft)
# #topic_model = BERTopic(embedding_model=word_doc_embedder)

In [91]:
import umap
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer

vectorizor_model = CountVectorizer(ngram_range=(1,3), stop_words=stopwords)

from sklearn.feature_extraction.text import TfidfVectorizer
#vectorizor_model = TfidfVectorizer(stop_words=stopwords,ngram_range=(1,1),max_df=0.6)

# Load sentence transformer model
#sentence_model = SentenceTransformer("roberta-base-nli-stsb-mean-tokens")
#embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embedding_model = SentenceTransformer("all-mpnet-base-v2")
#embedding_model = SentenceTransformer("msmarco-distilbert-base-tas-b")


#sentence_model = SentenceTransformer("msmarco-distilbert-base-tas-b")
# download sentence transformer here: https://huggingface.co/sentence-transformers

# Create documents embeddings
#embeddings = sentence_model.encode(texts, show_progress_bar=True)     

# Define UMAP model to reduce embeddings dimension
umap_model = umap.UMAP(n_neighbors=5,
                       n_components=5,
                       min_dist=0.05,
                       metric='cosine',
                       low_memory=False)

# Define HDBSCAN model to perform documents clustering
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=40,
                                min_samples=20,
                                metric='euclidean',
                                cluster_selection_method='leaf',#'eom',
                                gen_min_span_tree=True,
                                prediction_data=True)
# define Bertopic model
# ref: https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html#min_samples

from bertopic.vectorizers import ClassTfidfTransformer

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
topic_model = BERTopic(ctfidf_model=ctfidf_model )

topic_model = BERTopic(top_n_words=15,
                       nr_topics = 30,
                       n_gram_range=(1,3), 
                       calculate_probabilities=True,
                       umap_model= umap_model,
                       hdbscan_model=hdbscan_model,
                       embedding_model=embedding_model,#word_doc_embedder,#embedding_model,
                       vectorizer_model = vectorizor_model,
                       ctfidf_model=ctfidf_model,
                       #similarity_threshold_merging=0.5,
                       verbose=True,
                       language="english",
                       #calculate_probabilities=True,
                       min_topic_size=300 ) # better [100,500]


topics, probs = topic_model.fit_transform(df_train['review_en'].values)#, embeddings)
df_train_topic = pd.DataFrame({'topic': topics, 'document': df_train['review_en'].values})

topic_model.save("./model/bertopic_model_weights.h5")

Batches:   0%|          | 0/144 [00:00<?, ?it/s]

2022-10-04 09:53:48,315 - BERTopic - Transformed documents to Embeddings
2022-10-04 09:53:53,815 - BERTopic - Reduced dimensionality
2022-10-04 09:53:54,525 - BERTopic - Clustered reduced embeddings
2022-10-04 09:54:05,248 - BERTopic - Reduced number of topics from 35 to 31
c:\Users\nhan.ngothanh1\Anaconda3\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [92]:
# TOPIC_MODEL_PATH = './model/release_2022_10_03/'
# topic_model = BERTopic.load(TOPIC_MODEL_PATH+"bertopic_model_weights.h5")
# topics, probs = topic_model.transform(df_train['review_en'].values)#, embeddings)

In [93]:
df_train_topic = pd.DataFrame({'topic': topics, 'document': df_train['review_en'].values})
df_train_topic

,topic,document
0,9,why can not use coins app
1,0,I can not see the card info
2,-1,the application time item reported an error
3,-1,The card item in the myvib app has an error an...
4,13,error can not pay with coins
...,...,...
4582,10,smart otp feature not working
4583,-1,the app has no place to change the login passw...
4584,0,ask not to auto add credit insurance
4585,8,"Voice and AR features sound advanced, but are ..."


In [94]:
####################################################
# Create topic dictionary
####################################################

topic_dict = {}
topic_words = ['-1 : outlier']
topic_dict[-1] = 'outlier'
for i in range(len(set(topics))-1):
  tpc = topic_model.get_topic(i)[:15]
  words = [x[0] for x in tpc]
  topic_dict[i] = words
  tw = ' '.join([str(i) + ':'] + words)
  topic_words.append(tw)

# save topic dictionary to file
with open('./model/topic_dict.yaml', 'w') as f:
    data = yaml.dump(topic_dict, f, sort_keys=False, default_flow_style=False)


In [95]:
df_topic = pd.DataFrame()
df_topic['Topic'] = topic_dict.keys()
df_topic['keywords'] = [  '_'.join(topic_dict[i]) if (i!=-1) else topic_dict[i] for i in topic_dict.keys() ]
df_topic_info = topic_model.get_topic_info()
df_topic = pd.merge(df_topic,df_topic_info[['Topic','Count']],on='Topic', how='left')
#df_topic.to_csv('./model/topic_info.csv', index=False)
df_topic.head()


,Topic,keywords,Count
0,-1,outlier,1455
1,0,card_call_staff_bank_credit_fee_credit card_ca...,257
2,1,lag_laggy_slow_bugs_app lag_app laggy_buggy_ap...,193
3,2,kyc_crashing_keeps crashing_log_app keeps cras...,157
4,3,recharge_phone_top_recharge phone_top phone_mo...,143


In [96]:
topic_model.visualize_barchart()


In [97]:
topic_model.visualize_topics()

In [98]:
topic_model.visualize_hierarchy()

#### GROUP TOPIC

In [99]:
group_dict = {'G0': [19, 22],
 'G1': [18,2,14],
 'G2': [21, 28],
 'G3': [4, 13, 9,24],
 'G4': [5, 25,3],
 'G5': [6, 10],
 'G6': [1, 8, 27,17],
 'G7': [16, 29],
 'G8': [12, 7,15,26, 23],
 'G10':[11],
 'G11':[0,20],
 'UNK':[-1]}

group_dict_adj = {}
for g in group_dict.keys():
    for i in group_dict[g]:
        group_dict_adj[i] = g
#group_dict_adj



In [100]:
df_train_topic['group_topic'] = df_train_topic['topic'].apply(lambda x: group_dict_adj[x])
df_train_topic = pd.merge(df_train_topic, df_topic, left_on='topic', right_on='Topic', how='left')
df_train_topic = df_train_topic[['topic','document','group_topic','keywords']]
df_train_topic.to_csv("./train_data/train_data_with_topic_group.csv", index=False)
df_train_topic

,topic,document,group_topic,keywords
0,9,why can not use coins app,G3,qr_qr code_scan_coin_scan qr_code_coins_coin a...
1,0,I can not see the card info,G11,card_call_staff_bank_credit_fee_credit card_ca...
2,-1,the application time item reported an error,UNK,outlier
3,-1,The card item in the myvib app has an error an...,UNK,outlier
4,13,error can not pay with coins,G3,coin_payment_deducted_coin deducted_failed_pay...
...,...,...,...,...
4582,10,smart otp feature not working,G5,otp_smart otp_smart_otp code_code_battery_otp ...
4583,-1,the app has no place to change the login passw...,UNK,outlier
4584,0,ask not to auto add credit insurance,G11,card_call_staff_bank_credit_fee_credit card_ca...
4585,8,"Voice and AR features sound advanced, but are ...",G6,old_interface_version_old version_old app_app ...


In [101]:
# Manual Grouping and Name

group_desc_dict = {'G0':'relating to coin reward program (friend introduction)',
'G1': 'topup/bill payment',
'G10':'staff support service quality',
'G11':'relating to card',
'G2': 'relating to use, payment by coins',
'G3': 'relating to money transfer (can not transfer/slow transfer/transfered but not received)',
'G4':'login app and smart OTP issues',
'G5': 'hard ekyc registation, especially face identification',
'G6': 'quality of app (slow/lag/unattractive)',
'G7': 'app error complaint',
'G8': 'expressing disparage/disappoint to the app',
'UNK':'Outlier'}

In [102]:
df_topic_lookup = pd.DataFrame(data={'topic':group_dict_adj.keys()},columns=['topic'])
df_topic_lookup['gtopic'] =  df_topic_lookup['topic'].apply(lambda x: group_dict_adj[x])
df_topic_lookup['gtopic_content'] = df_topic_lookup['gtopic'].apply(lambda x: group_desc_dict[x])
df_topic_lookup.to_csv("./model/topic_lookup.csv", index=False)
df_topic_lookup

,topic,gtopic,gtopic_content
0,19,G0,relating to coin reward program (friend introd...
1,22,G0,relating to coin reward program (friend introd...
2,18,G1,topup/bill payment
3,2,G1,topup/bill payment
4,14,G1,topup/bill payment
5,21,G2,"relating to use, payment by coins"
6,28,G2,"relating to use, payment by coins"
7,4,G3,relating to money transfer (can not transfer/s...
8,13,G3,relating to money transfer (can not transfer/s...
9,9,G3,relating to money transfer (can not transfer/s...


In [103]:
df_topic_lookup.groupby(['gtopic','gtopic_content'])['topic'].count().reset_index()

,gtopic,gtopic_content,topic
0,G0,relating to coin reward program (friend introd...,2
1,G1,topup/bill payment,3
2,G10,staff support service quality,1
3,G11,relating to card,2
4,G2,"relating to use, payment by coins",2
5,G3,relating to money transfer (can not transfer/s...,4
6,G4,login app and smart OTP issues,3
7,G5,"hard ekyc registation, especially face identif...",2
8,G6,quality of app (slow/lag/unattractive),4
9,G7,app error complaint,2


In [104]:
df_train_topic['topic_content'] = df_train_topic['group_topic'].apply(lambda x: group_desc_dict[x])
df_train_topic.to_csv("./train_data/train_data_with_topic_group.csv", index=False)
df_train_topic

,topic,document,group_topic,keywords,topic_content
0,9,why can not use coins app,G3,qr_qr code_scan_coin_scan qr_code_coins_coin a...,relating to money transfer (can not transfer/s...
1,0,I can not see the card info,G11,card_call_staff_bank_credit_fee_credit card_ca...,relating to card
2,-1,the application time item reported an error,UNK,outlier,Outlier
3,-1,The card item in the myvib app has an error an...,UNK,outlier,Outlier
4,13,error can not pay with coins,G3,coin_payment_deducted_coin deducted_failed_pay...,relating to money transfer (can not transfer/s...
...,...,...,...,...,...
4582,10,smart otp feature not working,G5,otp_smart otp_smart_otp code_code_battery_otp ...,"hard ekyc registation, especially face identif..."
4583,-1,the app has no place to change the login passw...,UNK,outlier,Outlier
4584,0,ask not to auto add credit insurance,G11,card_call_staff_bank_credit_fee_credit card_ca...,relating to card
4585,8,"Voice and AR features sound advanced, but are ...",G6,old_interface_version_old version_old app_app ...,quality of app (slow/lag/unattractive)


In [105]:
# Save new model

import shutil
today = datetime.datetime.now()
if not os.path.exists('./model/dev_'+today.strftime('%Y_%m_%d')):
    os.mkdir('./model/dev_'+today.strftime('%Y_%m_%d'))

for file in ['topic_dict.yaml','topic_lookup.csv','bertopic_model_weights.h5']:
    shutil.copy('./model/'+file, './model/dev_'+today.strftime('%Y_%m_%d'))


### PREDICTION

In [106]:
# Load Topic Model
topic_model1 = BERTopic.load("./model/bertopic_model_weights.h5")

with open('./model/topic_dict.yaml') as f:
    topic_dict1 = yaml.load(f, Loader=SafeLoader)
    

In [107]:
reviews = ['the app is so bad, performance is quite slow',
           'login fails always, I am unhappy with the app',
           'supporting weak, waiting so long time to get the response from staff. I will never use the service any more.',
           'hard to use the app, the UI is not friendly. it makes me confused.',
           'cheating coupon campaign, i got nothing',
           'nothing to say about the service',
           'transfer money quite slow, near one day to get',
           'no more atm, inconvenient to withdraw. I can\'t any atm nearby.',
           'card is not recognized by machine. unconvenient',
           'can not payment online with the card',
           'wrong fee for credit card. please refund money back to me.',
           'call but not receive the relevant response from staff',
           'app get error in view profile information',
           'money transfer is slow, it takes long time for receiving.']

df_pred1 = pd.DataFrame(reviews, columns=['review'])

reviews = df_pred1['review'].values.tolist()
reviews = [str(i) for i in reviews]
reviews = [re.sub('\'ll',' will',i) for i in reviews]
print(reviews)
reviews = [str(i) for i in reviews]
reviews = [re.sub('\'ll',' will',i) for i in reviews]
reviews = [re.sub('don\'t','do not',i) for i in reviews]
reviews = [re.sub('can\'t','can not',i) for i in reviews]
reviews = [re.sub('hasn\'t','has not',i) for i in reviews]
reviews = [re.sub('wasn\'t','was not',i) for i in reviews]
reviews = [re.sub('won\'t','will not',i) for i in reviews]
reviews = [re.sub('cant','can not',i) for i in reviews]
reviews = [re.sub('dont','do not',i) for i in reviews]
reviews = [re.sub('hadn\'t','had not',i) for i in reviews]

# #-----------------------------------------
# # Lemmatizer
# from nltk import pos_tag
# from nltk.stem import WordNetLemmatizer
    
# reviews = [' '.join([WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tag(text.split())]) for text in reviews] 

# Keep Noun and Adj only
#reviews = [get_noun_adj(i) for i in reviews]
#reviews = [txti for txti in reviews if len(txti.split())>1]

# Keep Noun and Adj only
#reviews = [get_noun_adj(i) for i in reviews]

#embeddings_test = sentence_model.encode(reviews, show_progress_bar=False)
pred_rs = topic_model1.transform(reviews)#, embeddings_test)
#pred_rs = topic_model1.transform(reviews)
#print(pred_rs)
#print(pred_rs)
df_pred1['topic'] = pred_rs[0]
df_pred1['topic_content'] = df_pred1['topic'].apply(lambda x: topic_dict1[x] if x in topic_dict1.keys() else 'Outlier')
df_pred1

['the app is so bad, performance is quite slow', 'login fails always, I am unhappy with the app', 'supporting weak, waiting so long time to get the response from staff. I will never use the service any more.', 'hard to use the app, the UI is not friendly. it makes me confused.', 'cheating coupon campaign, i got nothing', 'nothing to say about the service', 'transfer money quite slow, near one day to get', "no more atm, inconvenient to withdraw. I can't any atm nearby.", 'card is not recognized by machine. unconvenient', 'can not payment online with the card', 'wrong fee for credit card. please refund money back to me.', 'call but not receive the relevant response from staff', 'app get error in view profile information', 'money transfer is slow, it takes long time for receiving.']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-10-04 09:54:21,574 - BERTopic - Reduced dimensionality
2022-10-04 09:54:21,580 - BERTopic - Calculated probabilities with HDBSCAN
2022-10-04 09:54:21,580 - BERTopic - Predicted clusters


,review,topic,topic_content
0,"the app is so bad, performance is quite slow",1,"[lag, laggy, slow, bugs, app lag, app laggy, b..."
1,"login fails always, I am unhappy with the app",2,"[kyc, crashing, keeps crashing, log, app keeps..."
2,"supporting weak, waiting so long time to get t...",-1,outlier
3,"hard to use the app, the UI is not friendly. i...",4,"[download, app bad, not download, download app..."
4,"cheating coupon campaign, i got nothing",14,"[like shit like, shit like, shit like shit, li..."
5,nothing to say about the service,-1,outlier
6,"transfer money quite slow, near one day to get",29,"[load, slow, slow money, slow money transfer, ..."
7,"no more atm, inconvenient to withdraw. I can't...",-1,outlier
8,card is not recognized by machine. unconvenient,0,"[card, call, staff, bank, credit, fee, credit ..."
9,can not payment online with the card,17,"[bill, payment, pay, not pay, bill payment, fa..."


In [108]:
topic_clus1 = [ i for i in df_pred1['topic'].unique() if i>=0]

for i in topic_clus1:
    print(i,':',topic_dict1[i])

1 : ['lag', 'laggy', 'slow', 'bugs', 'app lag', 'app laggy', 'buggy', 'app buggy', 'slow app', 'many bugs', 'lag app', 'app many bugs', 'app slow', 'bugs app', 'bug']
2 : ['kyc', 'crashing', 'keeps crashing', 'log', 'app keeps crashing', 'app keeps', 'register', 'keeps', 'crashes', 'app crashes', 'not register', 'not log', 'password', 'logged', 'app']
4 : ['download', 'app bad', 'not download', 'download app', 'not download app', 'errors', 'app bad not', 'app not', 'install', 'location', 'bad not', 'app', 'app like', 'app like shit', 'app not good']
14 : ['like shit like', 'shit like', 'shit like shit', 'like shit', 'like bad like', 'bad like', 'shit', 'like', 'like bad', 'many mistakes', 'mistakes', 'shit like bad', 'bad like shit', 'cheat like', 'many mistakes like']
29 : ['load', 'slow', 'slow money', 'slow money transfer', 'cant create link', 'create link', 'failed load', 'money transfer slow', 'load slow', 'long', 'transfer slow', 'take long', 'long time', 'cant create', 'transact

In [109]:
a = np.array(stopwords)
a

array({'both', 'her', 'their', 'other', 'needn', 'out', 'shouldn', 'itself', 'which', 'there', 'had', 'theirs', 'own', 'should', 'them', 'no', "couldn't", 'isn', 'application', "she's", 'such', 'now', 'can', "you're", 'also', 'must', 'it', 'some', 'when', 'through', 'for', 'were', 'down', 'into', "isn't", 'because', 'any', 'more', 'very', 'he', 'haven', 'are', 're', "needn't", 'have', "shan't", 'during', 'with', 'few', 'him', 'nor', 'has', 'couldn', 'our', 'its', 'd', 'wasn', 's', 'too', 've', 'am', 'himself', 'the', 'before', 'up', "weren't", 'in', 'again', 'won', 'doing', 'on', 'so', 'just', 'mightn', 'between', 'his', 'was', "you'd", 'here', 'regards', 'above', 'why', 'y', 'but', 'each', 'over', 'further', 'once', 'mustn', 'didn', 'then', 'whom', 'most', 'those', 'your', 'myself', "haven't", 'll', 'wouldn', 'as', 'will', "that'll", 'does', 'from', 'same', 'been', "shouldn't", 'while', "you've", 'aren', 'weren', "mustn't", 'i', "mightn't", 'by', 'below', 'and', 'yourself', 'she', 'do

#### Reference
https://www.youtube.com/watch?v=fb7LENb9eag
https://github.com/pinecone-io/examples/blob/master/learn/algos-and-libraries/bertopic/06_bertopic_params.ipynb
https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#word-document-embeddings
